In [1]:
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset
import os
from scipy import io
from torchvision.transforms import transforms
import torch
import cv2
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torchvision.transforms import transforms
import random
from PIL import Image
import numpy as np
import torch.optim as optim
import torchvision
from torchvision import models
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
class iouEval:

    def __init__(self, nClasses, device, ignoreIndex=-1):
        self.nClasses = nClasses
        self.ignoreIndex = ignoreIndex if nClasses>ignoreIndex else -1 #if ignoreIndex is larger than nClasses, consider no ignoreIndex
        self.reset()
        self.device = device

    def reset (self):
        classes = self.nClasses if self.ignoreIndex==-1 else self.nClasses-1
        self.tp = torch.zeros(classes).double()
        self.fp = torch.zeros(classes).double()
        self.fn = torch.zeros(classes).double()        

    def addBatch(self, x, y):   #x=preds, y=targets
        #sizes should be "batch_size x nClasses x H x W"
        
        #print ("X is cuda: ", x.is_cuda)
        #print ("Y is cuda: ", y.is_cuda)

        if (x.is_cuda or y.is_cuda):
            x = x.to(self.device)
            y = y.to(self.device)

        #if size is "batch_size x 1 x H x W" scatter to onehot
        if (x.size(1) == 1):
            x_onehot = torch.zeros(x.size(0), self.nClasses, x.size(2), x.size(3))  
            if x.is_cuda:
                x_onehot = x_onehot.to(self.device)
            x_onehot.scatter_(1, x, 1).float()
        else:
            x_onehot = x.float()

        if (y.size(1) == 1):
            y_onehot = torch.zeros(y.size(0), self.nClasses, y.size(2), y.size(3))
            if y.is_cuda:
                y_onehot = y_onehot.to(self.device)
            y_onehot.scatter_(1, y, 1).float()
        else:
            y_onehot = y.float()

        if (self.ignoreIndex != -1): 
            ignores = y_onehot[:,self.ignoreIndex].unsqueeze(1)
            x_onehot = x_onehot[:, :self.ignoreIndex]
            y_onehot = y_onehot[:, :self.ignoreIndex]
        else:
            ignores=0


        tpmult = x_onehot * y_onehot    #times prediction and gt coincide is 1
        tp = torch.sum(torch.sum(torch.sum(tpmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze()
        fpmult = x_onehot * (1-y_onehot-ignores) #times prediction says its that class and gt says its not (subtracting cases when its ignore label!)
        fp = torch.sum(torch.sum(torch.sum(fpmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze()
        fnmult = (1-x_onehot) * (y_onehot) #times prediction says its not that class and gt says it is
        fn = torch.sum(torch.sum(torch.sum(fnmult, dim=0, keepdim=True), dim=2, keepdim=True), dim=3, keepdim=True).squeeze() 

        self.tp += tp.double().cpu()
        self.fp += fp.double().cpu()
        self.fn += fn.double().cpu()

    def getIoU(self):
        num = self.tp
        den = self.tp + self.fp + self.fn + 1e-15
        iou = num / den
        return torch.mean(iou), iou     #returns "iou mean", "iou per class"

In [ ]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class Net(nn.Module):
    def __init__(self,NUM_CLASSES, p = 0.35):
        super(Net,self).__init__()

        self.base_model = models.resnet34(pretrained=True)
        self.base_layers = list(self.base_model.children())
        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample4 = nn.ConvTranspose2d(256,256,3,2,1,1)
        self.upsample3 = nn.ConvTranspose2d(128,128,3,2,1,1)
        self.upsample2 = nn.ConvTranspose2d(64,64,3,2,1,1)
        self.upsample1 = nn.ConvTranspose2d(64,64,3,2,1,1)
        self.upsample0 = nn.ConvTranspose2d(32,32,3,2,1,1)
        
        self.conv_up4 = convrelu(512 , 256 , 3 , 1)
        self.conv_up3 = convrelu(256 , 128, 3, 1)
        self.conv_up2 = convrelu(128, 64, 3, 1)
        self.conv_up1 = convrelu(64 , 64, 3, 1)
        self.conv_up0 = convrelu(64, 32, 3, 1)

        self.conv_original_size0 = convrelu(3, 32, 3, 1)
        self.conv_original_size1 = convrelu(32, 32, 3, 1)
        self.conv_original_size2 = convrelu(32, 16, 3, 1)

        self.conv_last = nn.Conv2d(16, NUM_CLASSES, 1)
        self.dropout = nn.Dropout(p) 

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)
        #print(x_original.shape)
        layer0 = self.layer0(input) 
        #print(layer0.shape)
        layer1 = self.layer1(layer0) #dropout can be added here inside
        
        layer2 = self.layer2(layer1) #dropout can be added here inside
        
        layer3 = self.layer3(layer2) #dropout can be added here inside
          
        layer4 = self.layer4(layer3) #dropout can be added here inside
        
        layer4 = self.dropout(self.layer4_1x1(layer4)) 
        
        layer4 = self.conv_up4(layer4) #256
        
        x = self.upsample4(layer4) #256
        layer3 = self.layer3_1x1(layer3) # 256
        #print('first concat:',x.shape,layer3.shape)
        x = x + layer3 #torch.cat([x, layer3], dim=1) #256
        
        #print(x.shape)
        x = self.conv_up3(x) # 128
        #print(x.shape)
        x = self.upsample3(x) # 128
        layer2 = self.layer2_1x1(layer2) #128
        #print('second concat', x.shape,layer2.shape)
        x = x + layer2#torch.cat([x, layer2], dim=1) # 128
        x = self.conv_up2(x) #64

        x = self.upsample2(x) #64
        layer1 = self.layer1_1x1(layer1) #64

        x = x + layer1 #torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x) #64

        x = self.upsample1(x) #64
        layer0 = self.layer0_1x1(layer0) #64

        x =  x + layer0 #torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x) #32

        x = self.upsample0(x) #32

        x = x + x_original#torch.cat([x, x_original], dim=1) #32
        x = self.conv_original_size2(x) #16

        out = self.conv_last(x) #6

        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

NUM_CLASSES = 14
OPT_LEARNING_RATE_INIT 	= 1e-3

OPT_BETAS 		= (0.9, 0.999)
OPT_EPS_LOW 		= 1e-08
OPT_WEIGHT_DECAY 	= 1e-4

image_path = '/content/drive/MyDrive/F1_datasets/13Labels/SUNRGBD-train_images' 
label_path = '/content/drive/MyDrive/F1_datasets/13Labels/train13labels'
savedir = '/content/drive/MyDrive/F1_datasets/13Labels'

In [ ]:
image_list = os.listdir(image_path)
image_list.sort()

label_list = os.listdir(label_path)
label_list.sort()

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# class_weights = torch.tensor()
ARGS_NUM_EPOCHS = 50
tran_batch_size = val_batch_size = 32

In [ ]:
Data = MyDataset(mean,std,image_path,label_path,label_list,image_list)
train_set, val_set = torch.utils.data.random_split(Data, [len(image_list) - 500, 500])
train_loader = DataLoader(train_set, batch_size=tran_batch_size,shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=val_batch_size,shuffle=True, num_workers=4)

In [ ]:
class CrossEntropyLoss2d(torch.nn.Module):
	def __init__(self, weight=None):
		super().__init__()
		self.loss = torch.nn.NLLLoss(weight=weight)

	def forward(self, outputs, targets,mask=None):
		return self.loss(torch.nn.functional.log_softmax(outputs, dim=1),targets)

# weight = class_weights.cuda()
# criterion = CrossEntropyLoss2d(weight=weight)
criterion = CrossEntropyLoss2d()
iou_best = 0
model = Net(NUM_CLASSES,0.2).to(device)

optimizer = optim.Adam(
  model.parameters(),
  OPT_LEARNING_RATE_INIT,
  OPT_BETAS,
  eps = OPT_EPS_LOW,
  weight_decay = OPT_WEIGHT_DECAY 
  )

In [ ]:
best_iou = 0

for epoch in range(ARGS_NUM_EPOCHS+1):
  print("\n ---------------- Epoch #", epoch, "------------------\n")
  epoch_loss = []
  iters = 0
  model.train()
  iouEvalTrain = iouEval(NUM_CLASSES,device)
  
  for step, (image,label) in enumerate(train_loader):

    iters += 1

    image = image.to(device)
    label = label.to(device)

    output = model(image)

    iouEvalTrain.addBatch(
      output.max(1)[1].unsqueeze(1).data,
      label.long().data
    )

    optimizer.zero_grad()
    loss = criterion(output,label[:,0].long())

    loss.backward()
    optimizer.step()
    epoch_loss.append(loss.item())
    
  
  avg_loss = sum(epoch_loss) / len(epoch_loss)
  iouTrain, iou_classes = iouEvalTrain.getIoU()

  print('[TRAINING] [Average loss]:{loss} [avg_iou]:{iou} [bg]:{bg} [roads]:{roads} [buildings]:{buildings} [vegetation]:{vegetation}'.format(
        loss = avg_loss,
        iou =  iouTrain))

  epoch_test_loss = []
  model.eval()
  iouEvalTest = iouEval(NUM_CLASSES,device)
  
  for step, (image,label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    output = model(image)

    iouEvalTest.addBatch(
      output.max(1)[1].unsqueeze(1).data,
      label.long().data
    )

    loss = criterion(output,label[:,0].long())
    epoch_test_loss.append(loss.item())
    
  
  avg_loss = sum(epoch_test_loss) / len(epoch_test_loss)
  iouTest, iou_classes = iouEvalTest.getIoU()

  

  print('[VALIDATION] [avg_val_loss]:{loss} [avg_iou]:{iou} [bg]:{bg} [roads]:{roads} [buildings]:{buildings} [vegetation]:{vegetation}'.format(
        loss = avg_loss,
        iou =  iouTest,
        bg = iou_classes[0]))
  
  if(iouTest > iou_best):
      iou_best = iouTest
      torch.save(model.state_dict(), savedir + '/model_best.pth')
      print('[SAVED] Best Model epoch:', epoch ,savedir +'/model_best.pth')